![Title](https://thenypost.files.wordpress.com/2019/03/shutterstock_790067248.jpg?quality=90&strip=all&w=618&h=410&crop=1)
# <div style="text-align: center" > A Statistical Analysis of airbnb in NYC</div>

I am supper excited to work with this airbnb dataset especially because this dataset is about NYC's airbnb hosts. I am going to know all the specs. This is going to be fun. Let's get started. 


# Goals:
* Do an exploratory EDA of Airbnb. 
* As a newyorker, I would like to know a bit more about the relation between Airbnb and NY neighbourhood.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
df = pd.read_csv('../input/new-york-city-airbnb-open-data/AB_NYC_2019.csv')

# Any results you write to the current directory are saved as output.

/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv
/kaggle/input/new-york-city-airbnb-open-data/New_York_City_.png


# About the Dataset

## Overview

In [2]:
# import pandas_profiling
# df.profile_report()

## Missing Values

In [3]:
def missing_percentage(df):
    """This function takes a DataFrame(df) as input and returns two columns, total missing values and total missing values percentage"""
    ## the two following line may seem complicated but its actually very simple. 
    total = df.isnull().sum().sort_values(ascending = False)[df.isnull().sum().sort_values(ascending = False) != 0]
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)[round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2) != 0]
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

missing_percentage(df)

Total  Percent
reviews_per_month  10052    20.56
last_review        10052    20.56
host_name             21     0.04
name                  16     0.03

These are missing values in this dataset. We will work with them later. 

## Memory Usage

In [4]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
id                                48895 non-null int64
name                              48879 non-null object
host_id                           48895 non-null int64
host_name                         48874 non-null object
neighbourhood_group               48895 non-null object
neighbourhood                     48895 non-null object
latitude                          48895 non-null float64
longitude                         48895 non-null float64
room_type                         48895 non-null object
price                             48895 non-null int64
minimum_nights                    48895 non-null int64
number_of_reviews                 48895 non-null int64
last_review                       38843 non-null object
reviews_per_month                 38843 non-null float64
calculated_host_listings_count    48895 non-null int64
availability_365                  48895 non-null int64

As you can see this is a fairly small dataset and there is no need to worry about reducing memory. However, as I am trying to learn about about this topic, I will demonstrate some techniques to reduce memory usages. First let's write a function with calculate the memory usage of the dataframe and try it on our dataset. 

In [5]:
def mem_usage(pandas_obj):
    """This function takes in a DataFrame object as an input and returns a"""
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

mem_usage(df)

'23.45 MB'

First we will work with "object" data types. 

## Object types

In [6]:
# This function takes only the 'object' type columns from the the dataFrames and the columns with tunique values 
def reduce_by_category_type(df):
    converted_obj = pd.DataFrame()
    for col in df.columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5 and df[col].dtype == 'object':
            converted_obj.loc[:,col] = df[col].astype('category')
        else:
            converted_obj.loc[:,col] = df[col]
    return converted_obj

df = reduce_by_category_type(df)

In [7]:
mem_usage(df)

'9.78 MB'

As you can see there is already a massive improvement in the dataset from 23.45 MB to 9.78 MB. Let's work with "int64" datatypes as well. 

In [8]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
id                                48895 non-null int64
name                              48879 non-null object
host_id                           48895 non-null int64
host_name                         48874 non-null category
neighbourhood_group               48895 non-null category
neighbourhood                     48895 non-null category
latitude                          48895 non-null float64
longitude                         48895 non-null float64
room_type                         48895 non-null category
price                             48895 non-null int64
minimum_nights                    48895 non-null int64
number_of_reviews                 48895 non-null int64
last_review                       38843 non-null category
reviews_per_month                 38843 non-null float64
calculated_host_listings_count    48895 non-null int64
availability_365                  48895 non-

In [9]:
df_int = df.select_dtypes(include = 'int64')
print (f" Before working with int types: {mem_usage(df_int)}")
converted_int = df.select_dtypes(include = 'int64').apply(pd.to_numeric,downcast='unsigned')
print (f" After working with the int types: {mem_usage(converted_int)}")
df.loc[:,df_int.columns] = converted_int
print (f" Total usage of the df after: {mem_usage(df)}")

 Before working with int types: 2.61 MB
 After working with the int types: 0.84 MB
 Total usage of the df after: 8.01 MB


So, the total improvement of the dataset is from 23.45 MB to 8.01 MB. This is a huge improvement.

# EDA

Let's have a preview of the dataset. 

In [10]:
df.head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  2018-10-19   
1  Entire home/apt    225               1                 45  2019-05-21   
2     Private room    150               3                  0         NaN   
3  Entire home/apt     89               1                270  2019-07-05   
4  Entire home/apt     80              10                  9  2018-11-19   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               0.21                               6               365  
1               0.38                               2               355  
2                NaN                               1               365  
3               4.64                               1               194  
4               0.10                               1                 0

In [11]:
df['adjusted_price'] = df.price/df.minimum_nights

In [12]:
df.head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  2018-10-19   
1  Entire home/apt    225               1                 45  2019-05-21   
2     Private room    150               3                  0         NaN   
3  Entire home/apt     89               1                270  2019-07-05   
4  Entire home/apt     80              10                  9  2018-11-19   

   reviews_per_month  calculated_host_listings_count  availability_365  \
0               0.21                               6               365   
1               0.38                               2               355   
2                NaN                               1               365   
3               4.64                               1               194   
4               0.10                               1                 0   

   adjusted_price  
0           149.0  
1           225.0  
2            50.0  
3            89.0  
4             8.0

## Top Hosts

In [13]:

import plotly.graph_objects as go

temp = df.host_id.value_counts().reset_index().head(20)
temp.columns = ['host_id', 'count']
temp = temp.merge(df[['host_id','host_name']], left_on = 'host_id',right_on = 'host_id', how = 'left', copy = False)
temp.drop_duplicates(inplace = True)

x = temp['host_name']
y = temp['count']

# Use the hovertext kw argument for hover text
fig = go.Figure(data=[go.Bar(x=x, 
                             y=y,
#                              hovertext=['27% market share', '24% market share', '19% market share'],
                            )])
# Customize aspect
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Top Airbnb hosts in NYC')
fig.layout.xaxis.title = 'Hosts'
fig.layout.yaxis.title = 'Host listings'
fig.show()

Let's look at the top 5 hosts in NYC and see where they are hosting in NYC.

In [14]:
import plotly.express as px
## Setting the DataFrame
temp = df.host_id.value_counts().reset_index().head(5)
temp.columns = ['host_id', 'count']
temp = temp.merge(df[['host_id','host_name','price', 'latitude','longitude']], left_on = 'host_id',right_on = 'host_id', how = 'left', copy = False)

## Setting up the Visualization..
fig = px.scatter_mapbox(temp, 
                        lat="latitude", 
                        lon="longitude", 
                        color="host_name", 
                        size="price",
#                         color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=30, 
                        opacity = .70,
                        zoom=11,
                       )
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or 
# "stamen-watercolor" yeild maps composed of raster tiles from various public tile servers which do 
# not require signups or access tokens
# fig.update_layout(mapbox_style="carto-positron", 
#                  )
fig.layout.mapbox.style = 'carto-positron'
fig.update_layout(title_text = 'Top 5 hosts and their hosted Locations<br>(Click legend to toggle hosts)', height = 800)

fig.show()

It looks like Manhattan is the preferable place for top hosts. Let's plot all the datapoints and see how that looks. 

In [15]:
df.head()

id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0     Private room    149               1                  9  2018-10-19   
1  Entire home/apt    225               1                 45  2019-05-21   
2     Private room    150               3                  0         NaN   
3  Entire home/apt     89               1                270  2019-07-05   
4  Entire home/apt     80              10                  9  2018-11-19   

   reviews_per_month  calculated_host_listings_count  availability_365  \
0               0.21                               6               365   
1               0.38                               2               355   
2                NaN                               1               365   
3               4.64                               1               194   
4               0.10                               1                 0   

   adjusted_price  
0           149.0  
1           225.0  
2            50.0  
3            89.0  
4             8.0

# Overview of all datapoints by Borough

In [16]:
## Setting up the Visualization..
fig = px.scatter_mapbox(df, 
                        hover_data = ['price','minimum_nights','room_type'],
                        hover_name = 'neighbourhood',
                        lat="latitude", 
                        lon="longitude", 
                        color="neighbourhood_group", 
                        size="price",
#                         color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=30, 
                        opacity = .70,
                        zoom=10,
                       )
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or 
# "stamen-watercolor" yeild maps composed of raster tiles from various public tile servers which do 
# not require signups or access tokens
# fig.update_layout(mapbox_style="carto-positron", 
#                  )
fig.layout.mapbox.style = 'stamen-terrain'
fig.update_layout(title_text = 'Airbnb by Borough in NYC<br>(Click legend to toggle borough)', height = 800)
fig.show()

In [17]:
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
labels = df.neighbourhood_group.value_counts().index
values = df.neighbourhood_group.value_counts().values

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_traces(marker=dict(colors = colors, line=dict(color='#000000', width=2)))

fig.show()

It looks like Manhattan and Brooklyn counts for ~85% of all hostings.  

# How about by Neighbourhood

In [18]:
## Setting up the Visualization..
fig = px.scatter_mapbox(df, 
                        lat="latitude", 
                        lon="longitude", 
                        color="neighbourhood", 
#                         size="price",
#                         color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=30, 
                        opacity = .70,
                        zoom=10,
                       )
# "open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" or 
# "stamen-watercolor" yeild maps composed of raster tiles from various public tile servers which do 
# not require signups or access tokens
# fig.update_layout(mapbox_style="carto-positron", 
#                  )
fig.layout.mapbox.style = 'carto-positron'
fig.update_layout(title_text = 'NYC Airbnb by Neighbourhood<br>(Click legend to toggle neighbourhood)', height = 800)

fig.show()

# Room Types

In [19]:
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
labels = df.room_type.value_counts().index
values = df.room_type.value_counts().values

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_traces(marker=dict(colors = colors, line=dict(color='#000000', width=2)))

fig.show()

In [20]:
from plotly.subplots import make_subplots

temp_bk = df[df.neighbourhood_group == 'Brooklyn']
temp_qn = df[df.neighbourhood_group == 'Queens']
temp_mn = df[df.neighbourhood_group == 'Manhattan']


labels = df.room_type.value_counts().index.to_list()

fig = make_subplots(1, 3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Manhattan', 'Brooklyn', 'Queens'])
fig.add_trace(go.Pie(labels=labels, values=temp_mn.room_type.value_counts().reset_index().sort_values(by = 'index').room_type.tolist(), scalegroup='one',
                     name="Manhattan"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=temp_bk.room_type.value_counts().reset_index().sort_values(by = 'index').room_type.tolist(), scalegroup='one',
                     name="Brooklyn"), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=temp_qn.room_type.value_counts().reset_index().sort_values(by = 'index').room_type.tolist(), scalegroup='one',
                     name="Brooklyn"), 1, 3)


fig.update_layout(title_text='Room Types in top 3 Boroughs')
fig.update_traces(marker=dict(colors = colors, line=dict(color='#000000', width=2)))
fig.show()